In [1]:
import pandas as pd
import numpy as np 

In [5]:
!pip install openpyxl

   ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.0 kB 660.6 kB/s eta 0:00:01
   ------------------- -------------------- 122.9/250.0 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 250.0/250.0 kB 3.1 MB/s eta 0:00:00


In [2]:
df = pd.read_excel(r'C:\Users\User\Desktop\NIR\База данных МРТ гипофиза (1).xlsx')

In [3]:
df['Файл с разметкой Ax T1+C'].isna().value_counts()

Файл с разметкой Ax T1+C
False    72
True     34
Name: count, dtype: int64

# Скрипт для загрузки с диска

In [4]:
import os
import pandas as pd
import requests
import zipfile
import shutil
from urllib.parse import urlencode
#Assuming your dataframe is df


# Создание папки если она не существует
data_dir = 'data_29_05_24_cls108_seg72'
# os.makedirs(data_dir, exist_ok=True)

# Обход DataFrame
for index, row in df.iterrows():
    patient_id = row['ID пациента'] 
    patient_dir = f'{data_dir}/ID_{patient_id}' # путь к директории пациента

    # Создание папки для пациента, если она не существует
    os.makedirs(patient_dir, exist_ok=True)


    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    public_key = row['Местоположение файлов']  # Сюда вписываете вашу ссылку

    # Получаем загрузочную ссылку
    final_url = base_url + urlencode(dict(public_key=public_key))
    # Скачивание и разархивирование файла Если это прямые ссылки на zip файлы
    # response = requests.get(final_url, stream=True)
    response = requests.get(final_url)
    download_url = response.json()['href']
    zip_file_path = os.path.join(patient_dir, 'temp.zip') # Путь к временному zip файлу
    download_response = requests.get(download_url)

    # Сохранение zip файла
    with open(zip_file_path, 'wb') as fd:
        fd.write(download_response.content)

    # Разархивация файла
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(patient_dir)

    # Удаление временного zip файла
    os.remove(zip_file_path)
    
    # # Создание файла с метками класса
    with open(os.path.join(patient_dir, 'labels.txt'), 'w', encoding='utf-8') as f:
        f.write(f'Категория Knosp слева (S): {row["Категория Knosp слева (S)"]}\n')
        f.write(f'Категория Knosp справа (D): {row["Категория Knosp справа (D)"]}\n')

In [ ]:
lables = {
    '0':0,
    '1':1,
    '2':2,
    '3a':3,
    '3b':4,
    '4':5
}